In [2]:
import os
import numpy as np
import cv2 as cv

In [3]:
def inside(r, q):
    rx, ry, rw, rh = r
    qx, qy, qw, qh = q
    return rx > qx and ry > qy and rx + rw < qx + qw and ry + rh < qy + qh


def draw_detections(img, rects, thickness = 1):
    for x, y, w, h in rects:
        # the HOG detector returns slightly larger rectangles than the real objects.
        # so we slightly shrink the rectangles to get a nicer output.
        pad_w, pad_h = int(0.15*w), int(0.05*h)
        cv.rectangle(img, (x+pad_w, y+pad_h), (x+w-pad_w, y+h-pad_h), (0, 255, 0), thickness)


In [4]:
def peopledetect(folder):
    import sys
    from glob import glob
    import itertools as it

    # print(__doc__)

    hog = cv.HOGDescriptor()
    hog.setSVMDetector( cv.HOGDescriptor_getDefaultPeopleDetector() )

    #default = ['../data/basketball2.png '] if len(sys.argv[1:]) == 0 else []

    #for fn in it.chain(*map(glob, default + sys.argv[1:])):
    for fn in os.listdir(folder):
        #print(fn, ' - ',)
        try:
            img = cv.imread(folder + '/' + fn)
            if img is None:
                print('Failed to load image file:', fn)
                continue
        except:
            print('loading error')
            continue

        found, w = hog.detectMultiScale(img, winStride=(8,8), padding=(32,32), scale=1.05)
        found_filtered = []
        for ri, r in enumerate(found):
            for qi, q in enumerate(found):
                if ri != qi and inside(r, q):
                    break
            else:
                found_filtered.append(r)
        draw_detections(img, found)
        draw_detections(img, found_filtered, 3)
        print('%d (%d) found' % (len(found_filtered), len(found)))
        cv.imshow('img', img)
        ch = cv.waitKey()
        if ch == 27:
            break
    cv.destroyAllWindows()

In [5]:
peopledetect('pedestrian')

0 (0) found
4 (4) found
0 (0) found
7 (8) found
8 (8) found
